https://stackoverflow.com/questions/276052/how-to-get-current-cpu-and-ram-usage-in-python

https://pypi.org/project/psutil/<BR>
https://psutil.readthedocs.io/en/latest/

In [ ]:
!pip install psutil

In [ ]:
import requests
import json
import subprocess
import psutil
import pickle
import os
import time
import datetime
import pandas
import matplotlib.pyplot as plt

# one-time system details

In [ ]:
print(psutil.cpu_freq())

In [ ]:
pid = os.getpid()
print(pid)

In [ ]:
py = psutil.Process(pid)
print(py)

In [ ]:
py.name()

In [ ]:
py.exe()

In [ ]:
py.cwd()

In [ ]:
py.cmdline()

# system performance

In [ ]:
# gives a single float value
psutil.cpu_percent(percpu=True)

In [ ]:
psutil.cpu_times_percent(percpu=True)

In [ ]:
for reslt in psutil.cpu_times_percent(percpu=True):
    print(dict(reslt._asdict()))

In [ ]:
# gives an object with many fields
psutil.virtual_memory()

In [ ]:
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

In [ ]:
dict(psutil.swap_memory()._asdict())

In [ ]:
dict(psutil.cpu_stats()._asdict())

In [ ]:
psutil.disk_io_counters(perdisk=True).keys()

In [ ]:
dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict())

In [ ]:
psutil.sensors_temperatures()

In [ ]:
psutil.sensors_fans()

In [ ]:
psutil.sensors_battery()

### put everything into a single row

In [ ]:
def row_for_system_performance(row):
    if not isinstance(row,dict):
        print("ERROR: expected a dict as input")
        return None
    cpu_pct_list = psutil.cpu_percent(percpu=True)
    for indx in range(len(cpu_pct_list)):
        row['CPU percent for '+str(indx)]=cpu_pct_list[indx]
    
    row['time in unix seconds']=time.time()
    row['datetime']=datetime.datetime.utcfromtimestamp(row['time in unix seconds'])

    indx=0
    for reslt in psutil.cpu_times_percent(percpu=True):
        indx+=1
        for k,v in dict(reslt._asdict()).items():
            row['CPU '+str(indx)+' times '+k]=v
    for disk_label in psutil.disk_io_counters(perdisk=True).keys():
        for k,v in dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict()).items():
            row['disk '+disk_label+' '+k]=v
    for k,v in dict(psutil.cpu_stats()._asdict()).items():
        row['CPU stats '+k]=v
    for k,v in dict(psutil.virtual_memory()._asdict()).items():
        row['system virtual memory '+k]=v
    for k,v in dict(psutil.swap_memory()._asdict()).items():
        row['system swap memory '+k]=v
    return row

# performance of current process

In [ ]:
os.getpid()

In [ ]:
type(os.getpid())

In [ ]:
py = psutil.Process(os.getpid())
print(py)

In [ ]:
dict(py.memory_info()._asdict())

In [ ]:
memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
print('memory use:', memoryUse,'GB')

In [ ]:
dict(py.memory_full_info()._asdict())

In [ ]:
py.cpu_percent()

In [ ]:
dict(py.cpu_times()._asdict())

In [ ]:
py.cmdline()

In [ ]:
dict(py.num_ctx_switches()._asdict())

In [ ]:
dict(py.io_counters()._asdict())

In [ ]:
def row_for_performance_per_pid(row,pid):
    if not isinstance(row,dict):
        print("ERROR: expected a dict as input for row")
        return None
    if not isinstance(pid,int):
        print("ERROR: expected a int as input for pid")
        return None
    for k,v in dict(psutil.Process(pid).memory_full_info()._asdict()).items():
        row['process memory: '+k]=v
    for k,v in dict(py.cpu_times()._asdict()).items():
        row['process cpu times: '+k]=v
    for k,v in dict(py.num_ctx_switches()._asdict()).items():
        row['process number context switches: '+k]=v
    for k,v in dict(py.io_counters()._asdict()).items():
        row['process io counters: '+k]=v
    return row

# gather data and create plots

In [ ]:
list_of_rows=[]
start_time = time.time()
for indx in range(10):
    row = {}
    row = row_for_system_performance(row)
    row = row_for_performance_per_pid(row,os.getpid())
    list_of_rows.append(row)
    time.sleep(1)

print('elapsed time:',time.time()-start_time,'seconds')

df = pandas.DataFrame(list_of_rows)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.to_pickle("single_process_df_this_notebook_"+str(time.time())+".pkl")

#df = pandas.read_pickle("single_process_df_this_notebook_1552783848.2168732.pkl")

In [ ]:
for col_name in df.columns:
    if 'seconds' not in col_name:
#        plt.scatter(x=df['time in unix seconds'],y=df[col_name])
        plt.plot_date(x=df['datetime'],y=df[col_name])
        plt.xticks(rotation=40)
        _=plt.ylabel(col_name,fontsize=14)
        _=plt.xlabel('time',fontsize=14)
        plt.show()
    else:
        print(col_name)

# track other notebooks

Plan: associate pid with each notebook kernel ID, then track the stats for that notebook by pid

### get a list of all active sessions

In [ ]:
r = requests.get('http://localhost:8888/api/sessions')
r.status_code

In [ ]:
list_of_kernel_dicts = r.json()
list_of_kernel_dicts[0]

### from json returned from API, we can extract the notebook kernel IDs

In [ ]:
print('notebook kernel IDs:')
for notebook_indx in range(len(list_of_kernel_dicts)):
    print(list_of_kernel_dicts[notebook_indx]['kernel']['id'])

### get a list of all pids

In [ ]:
print('all processes running:')
for p in psutil.process_iter():
    print(p)

### get pid for each notebook

In [ ]:
dict_of_jupyter_kernel_pids = {}
for p in psutil.process_iter():
    list_of_commands_per_process = psutil.Process(p.pid).cmdline()
#    print(list_of_commands_per_process)
    if ('ipykernel_launcher' in list_of_commands_per_process):
        dict_of_jupyter_kernel_pids[p.pid] = list_of_commands_per_process[len(list_of_commands_per_process)-1]
        
dict_of_jupyter_kernel_pids

### (alternative method) use `ps -ax` to get the relevant pid for notebooks

not in use

### add pid to dict containing kernel information

In [ ]:
for pid,kernel_path in dict_of_jupyter_kernel_pids.items():
    for notebook_indx in range(len(list_of_kernel_dicts)):
        if (list_of_kernel_dicts[notebook_indx]['kernel']['id'] in kernel_path):
#            print(pid,'is',list_of_kernel_dicts[notebook_indx],'\n')
            list_of_kernel_dicts[notebook_indx]['pid']=pid

In [ ]:
list_of_kernel_dicts

### which kernels are not this notebook?

In [ ]:
list_of_kernel_dicts_for_other_notebooks=[]
for kernel_dict in list_of_kernel_dicts:
    if (str(os.getpid())!=str(kernel_dict['pid'])):
        list_of_kernel_dicts_for_other_notebooks.append(kernel_dict)

In [ ]:
list_of_kernel_dicts_for_other_notebooks

# collect data for other notebook and create plots

In [ ]:
nb_dict={}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_dict[other_nb['pid']] = []

start_time = time.time()
for indx in range(10):
    row = {}
    system_row = row_for_system_performance(row)
    for other_nb in list_of_kernel_dicts_for_other_notebooks:
        row = row_for_performance_per_pid(system_row,int(other_nb['pid']))
        nb_dict[other_nb['pid']].append(row)
    time.sleep(1)

print('elapsed time:',time.time()-start_time,'seconds')

nb_df = {}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_df[other_nb['pid']] = pandas.DataFrame(nb_dict[other_nb['pid']])

In [ ]:
nb_df.keys()

In [ ]:
df = nb_df[list(nb_df.keys())[0]]
df.head()

In [ ]:
for col_name in df.columns:
    if 'seconds' not in col_name:
#        plt.scatter(x=df['time in unix seconds'],y=df[col_name])
        plt.plot_date(x=df['datetime'],y=df[col_name])
        plt.xticks(rotation=40)
        _=plt.ylabel(col_name,fontsize=14)
        _=plt.xlabel('time',fontsize=14)
        plt.show()
    else:
        print(col_name)